In [1]:
# multivariate multi-step stacked lstm example
import numpy as np
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import pandas as pd
import pickle as pkl

In [2]:
# NTHU
nthu_scaled = pd.read_pickle('../input/ddd-final-datasets/final_step2_scaled.pkl')

# get video info from other dataset
nthu_UNscaled = pd.read_pickle('../input/ddd-final-datasets/final_step2.pkl')
video_info_df = nthu_UNscaled.iloc[:,0:4]

# and concat video info with scaled df
nthu = pd.concat([video_info_df, nthu_scaled], axis=1)

# fix drowsiness column
nthu["DROWSINESS"] = nthu.loc[:, "DROWSINESS"].map({0:0, 1:2})

In [3]:
# add nthu videos to lists
g = nthu.groupby(["subject", "external_factors", "facial_actions"])
len(g.groups.keys())

nthu_train_list = []
nthu_test_list = []

for key in g.groups.keys():
    print(key)
    if key[0] in ['004', '022', '026', '030']:
        nthu_test_list.append(g.get_group(key).iloc[:,4:])
    else:
        nthu_train_list.append(g.get_group(key).iloc[:,4:])

('001', 'glasses', 'nonsleepyCombination')
('001', 'glasses', 'sleepyCombination')
('001', 'glasses', 'slowBlinkWithNodding')
('001', 'glasses', 'yawning')
('001', 'night_noglasses', 'nonsleepyCombination')
('001', 'night_noglasses', 'sleepyCombination')
('001', 'night_noglasses', 'slowBlinkWithNodding')
('001', 'night_noglasses', 'yawning')
('001', 'nightglasses', 'nonsleepyCombination')
('001', 'nightglasses', 'sleepyCombination')
('001', 'nightglasses', 'slowBlinkWithNodding')
('001', 'nightglasses', 'yawning')
('001', 'noglasses', 'nonsleepyCombination')
('001', 'noglasses', 'sleepyCombination')
('001', 'noglasses', 'slowBlinkWithNodding')
('001', 'noglasses', 'yawning')
('002', 'glasses', 'nonsleepyCombination')
('002', 'glasses', 'sleepyCombination')
('002', 'glasses', 'slowBlinkWithNodding')
('002', 'glasses', 'yawning')
('002', 'night_noglasses', 'nonsleepyCombination')
('002', 'night_noglasses', 'sleepyCombination')
('002', 'night_noglasses', 'slowBlinkWithNodding')
('002', 'n

In [4]:
# split a multivariate sequence into samples
def split_sequences(dataset_arr, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(dataset_arr)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(dataset_arr) or end_ix > len(dataset_arr):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = dataset_arr[i:end_ix, :-1], dataset_arr[end_ix:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [5]:
# split sequences for nthu train
nthu_train_seq_list_X=[]
nthu_train_seq_list_y=[]

n_steps_in, n_steps_out = 150, 60

for video_df in nthu_train_list:
    video_arr=np.array(video_df)
    # covert into input/output
    X, y = split_sequences(video_arr, n_steps_in, n_steps_out)
    if len(X)==0 or len(y)==0:
        continue
    nthu_train_seq_list_X.append(X)
    nthu_train_seq_list_y.append(y)

# concat lists
nthu_train_X = np.concatenate(nthu_train_seq_list_X, axis=0)
nthu_train_y = np.concatenate(nthu_train_seq_list_y, axis=0)

pkl.dump(nthu_train_X, open("nthu_train_X", 'wb'))
pkl.dump(nthu_train_y, open("nthu_train_y", 'wb'))

In [6]:
# split sequences for nthu test
nthu_test_seq_list_X=[]
nthu_test_seq_list_y=[]

n_steps_in, n_steps_out = 150, 60

for video_df in nthu_test_list:
    video_arr=np.array(video_df)
    # covert into input/output
    X, y = split_sequences(video_arr, n_steps_in, n_steps_out)
    if len(X)==0 or len(y)==0:
        continue
    nthu_test_seq_list_X.append(X)
    nthu_test_seq_list_y.append(y)

# concat lists
nthu_test_X = np.concatenate(nthu_test_seq_list_X, axis=0)
nthu_test_y = np.concatenate(nthu_test_seq_list_y, axis=0)

pkl.dump(nthu_test_X, open("nthu_test_X", 'wb'))
pkl.dump(nthu_test_y, open("nthu_test_y", 'wb'))

# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)